In [1]:
#imports
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

Since we are looking for classification models we will look at these 5:\
Naïve Bayes\
k-Nearest Neighbor\
Decision Trees\
Support Vector Machine\
Neural Networks

First we will Gridsearch for best params for each model for highest score

In [2]:
train = pd.read_csv('train_cleaned.csv', index_col=0)
valid= train.pop('Survived')
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train, valid, test_size=0.3, random_state=0) 


In [58]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

gnb = GaussianNB()
n_range = [i for i in range(1,100)]
gnb_params= [{'var_smoothing': np.logspace(0,-9, num=100)}]

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn_params = [{'n_neighbors':list(range(1,40)),
               'weights':['uniform','distance'],
               'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
               'leaf_size': n_range,
               'metric': ['euclidean' , 'manhattan' , 'chebyshev']   
              }]

dtc = DecisionTreeClassifier()
dtc_params = [{
               'criterion' : ['gini', 'entropy'], 
                'splitter' : ['best','random'],
               'max_depth': [2,3,4,5,6,7,8,9,10,12]
              }]

svc= SVC()
svc_params = [{'kernel':['linear', 'poly', 'rbf', 'sigmoid'] , 'degree': [1,2,3,4,5,6,7,8,9],
              'gamma' : ['scale','auto'], 'shrinking' : [True, False], 'decision_function_shape' :['ovo','ovr']
              }]

mlp = MLPClassifier()
# remove lbfgs solver as its for smaller datasets
mlp_params= [{
    'activation': ['tanh', 'relu','identity','logistic'],
    'solver': ['sgd', 'adam'],
    'alpha': [.0001, .0002, .0003, .0004, .00005, .00006,0.001,0.01,0.1, 0.05],
    'learning_rate': ['constant','adaptive'],
    'max_iter' : [5000],
    'hidden_layer_sizes': [
             (1,),(2,),(3,),(4,),(5,),(6,),(7,),(8,),(9,),(10,),(11,), (12,),(13,),(14,),(15,),(16,),(17,),(18,),(19,),(20,),(21,)
             ],
    'random_state': [0]
}]

lr = LogisticRegression()

lr_params = [{'penalty': ['l1', 'l2', 'elasticnet', 'none'], 
             'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}]

rf = RandomForestClassifier()
rf_params = {
    'n_estimators': [100,200,500,750,1000],
    'max_depth': [3,5,7,9]
}



In [4]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.get_params().keys()

dict_keys(['priors', 'var_smoothing'])

In [5]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.get_params().keys()

dict_keys(['algorithm', 'leaf_size', 'metric', 'metric_params', 'n_jobs', 'n_neighbors', 'p', 'weights'])

In [6]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.get_params().keys()

dict_keys(['ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'random_state', 'splitter'])

In [54]:
rf.get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [59]:
from tqdm import tqdm
models = {gnb : gnb_params, knn:knn_params,dtc:dtc_params, svc:svc_params,mlp:mlp_params,lr:lr_params ,rf:rf_params}
scores = {}
for i in tqdm(models):
    grid=GridSearchCV(i,models[i], scoring= 'accuracy',verbose = 2 )
    grid.fit(x_train, y_train)
    print ('for model', i , 'best estimator:' , grid.best_estimator_ , '\nbest score:\n',grid.best_score_,'\nbest params:\n', 
           grid.best_params_)
    scores[i]=  grid.cv_results_


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END ......................max_depth=3, n_estimators=100; total time=   0.2s
[CV] END ......................max_depth=3, n_estimators=100; total time=   0.0s
[CV] END ......................max_depth=3, n_estimators=100; total time=   0.0s
[CV] END ......................max_depth=3, n_estimators=100; total time=   0.0s
[CV] END ......................max_depth=3, n_estimators=100; total time=   0.0s
[CV] END ......................max_depth=3, n_estimators=200; total time=   0.2s
[CV] END ......................max_depth=3, n_estimators=200; total time=   0.1s
[CV] END ......................max_depth=3, n_estimators=200; total time=   0.2s
[CV] END ......................max_depth=3, n_estimators=200; total time=   0.2s
[CV] END ......................max_depth=3, n_estimators=200; total time=   0.2s
[CV] END ......................max_depth=3, n_estimators=500; total time=   0.5s
[CV] END ......................max_depth=3, n_e

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:08<00:00, 68.02s/it]

[CV] END .....................max_depth=9, n_estimators=1000; total time=   1.3s
for model RandomForestClassifier() best estimator: RandomForestClassifier(max_depth=3) 
best score:
 0.8298709677419355 
best params:
 {'max_depth': 3, 'n_estimators': 100}


gnb : gnb_params, knn:knn_params,dtc:dtc_params, svc:svc_params,\
for model GaussianNB() best estimator: GaussianNB(var_smoothing=0.12328467394420659) \
best score:\
 0.7993290322580646 \
best params:\
 {'var_smoothing': 0.12328467394420659}\
 
for model KNeighborsClassifier() best estimator: KNeighborsClassifier(leaf_size=4, metric='manhattan', n_neighbors=16) \
best score:\
 0.8073548387096775 \
best params:\
 {'algorithm': 'auto', 'leaf_size': 4, 'metric': 'manhattan', 'n_neighbors': 16, 'weights': 'uniform'}\
 
for model DecisionTreeClassifier() best estimator: DecisionTreeClassifier(criterion='entropy', max_depth=5) \
best score:\
 0.8266709677419355 \
best params:\
 {'criterion': 'entropy', 'max_depth': 5, 'splitter': 'best'}\
 
 for model SVC() best estimator: SVC(decision_function_shape='ovo', degree=2, kernel='poly') \
best score:\
 0.8282967741935483 \
best params:\
 {'decision_function_shape': 'ovo', 'degree': 2, 'gamma': 'scale', 'kernel': 'poly', 'shrinking': True}\
 
 for model MLPClassifier() best estimator: MLPClassifier(alpha=0.1, hidden_layer_sizes=(7,), max_iter=5000, random_state=0) \
best score:\
 0.8314709677419355 \
best params:\
 {'activation': 'relu', 'alpha': 0.1, 'hidden_layer_sizes': (7,), 'learning_rate': 'constant', 'max_iter': 5000, \'random_state': 0, 'solver': 'adam'}\
 
 for model LogisticRegression() best estimator: LogisticRegression(penalty='l1', solver='saga') \
best score:\
 0.8042322580645163 \
best params:\
 {'penalty': 'l1', 'solver': 'saga'}\

The best score came from Multi-layer Perceptron Classifier with a score of 0.83 , so we will move forward with this model
## Training and testing with this model


In [10]:
#parmas = {'activation': 'relu', 'alpha': 0.1, 'hidden_layer_sizes': (7,), 
# 'learning_rate': 'constant', 'max_iter': 5000, 'random_state': 0, 
# 'solver': 'adam'}

model =  MLPClassifier(activation = 'relu', alpha = 0.1 , hidden_layer_sizes= (7,),
                      learning_rate = 'constant', max_iter= 5000 , random_state = 0 
                      ).fit(x_train,y_train)

Taking a look at the accuracy of the model

In [13]:
model.score(x_test,y_test)
y_pred = model.predict(x_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)


0.8134328358208955

It's lower than we want it to be but we will work with it

## Applying model onto test set

In [20]:
test = pd.read_csv('test_cleaned.csv',index_col=0)
passengerid = test.pop('PassengerId')
test.head()

,Sex,Age,SibSp,Parch,Fare,Pclass_2,Pclass_3,Embarked_Q,Embarked_S
0,1,0.334993,-0.499470,-0.400248,-0.498407,0,1,1,0
1,0,1.325530,0.616992,-0.400248,-0.513274,0,1,0,1
2,1,2.514175,-0.499470,-0.400248,-0.465088,1,0,1,0
3,1,-0.259330,-0.499470,-0.400248,-0.483466,0,1,0,1
4,0,-0.655545,0.616992,0.619896,-0.418471,0,1,0,1


In [48]:
test_pred = model.predict(test)
predicted_df = pd.DataFrame(test_pred)
predicted_df.rename(columns = {0:'Survived'}, 
            inplace = True)
final_df = pd.concat([passengerid,predicted_df],axis=1)
final_df.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


Convert and submit

In [50]:
final_df.to_csv('submission.csv',index=False)